In [3]:
import polars as pl
import numpy as np

In [6]:
data = pl.read_csv("./output_hybrib_search.csv", encoding='iso-8859-1', use_pyarrow=True)

In [42]:
from ollama import chat
from pydantic import BaseModel
from typing import List
import tqdm

class Verify(BaseModel):
    matching: int

verify = []

for i in tqdm.tqdm(range(data.shape[0]), colour='green'):
    
    query = data['Question'][i]
    ground_truth = data['Sample'][i]
    hybrid_search = data['AnswerbyHybribsearch'][i]
    
    
    context = f''' 
        I have a question: {query}

        This is the correct answer: {ground_truth}

        This is the predicted answer: {hybrid_search}

        Please help me verify whether the correct answer and the predicted answer match based on the keywords in both type of answers.

        If they match, return 1; otherwise, return 0.
    '''

    response = chat(
        messages=[
            {
                'role': 'user',
                'content': context,
            }
        ],
        model='llama3.2:latest',
        format=Verify.model_json_schema(),
    )

    result = Verify.model_validate_json(response.message.content)
    verify.append(result.matching)
    
    df_results = pl.DataFrame({
        'query': data['Question'][:i+1],
        'ground_truth': data['Sample'][:i+1],
        'predict_from_hybrid_search': data['AnswerbyHybribsearch'][:i+1],
        'score_matching': verify
    })

    
    df_results.write_csv("./hani_hybrid_verified.csv")


100%|██████████| 1744/1744 [1:15:22<00:00,  2.59s/it]


In [46]:
df_results.filter(
    pl.col('score_matching').is_in([0, 1])
)['score_matching'].value_counts()

score_matching,count
i64,u32
1,457
0,1024


In [26]:
import requests
import json

def extract_keyword_by_llama32(query:str):
  
  
  url = 'http://localhost:11434/api/chat'

  context = f'''
  Extract the most relevant keywords from the following sentence: '{query}'. 
  Focus on important nouns, verbs, and adjectives that convey the core meaning. 
  Additionally, categorize the query into a relevant topic or section (e.g., Technology, Business, Health, Education, etc.).
  For each extracted keyword, provide a list of synonyms that accurately reflect its meaning in the given context. 
  Ensure that the synonyms include common alternatives, domain-specific terms (if applicable), 
  and variations in different registers (formal, informal, technical, etc.).
  '''

  headers = {
      "Content-Type": "application/json"
  }

  messages = [
      {"role": "system", "content": "You are an AI assistant trained to extract keywords and categorize queries."},
      {"role": "user", "content": context}
  ]

  model='llama3.2:latest'

  format={
      "type": "object",
      "properties": {
        "keywords": {
          "type": "array",
          'items': {
            'type': 'string'
          }
        }
      },
      "required": [
        "keywords"
      ]
    }

  data = {
      "model": model,
      "messages": messages,
      "format": format,
      "stream": False
  }

  response = requests.post(url, headers=headers, data=json.dumps(data))

  if response.status_code == 200:
      try:
          print(response.json())
      except json.JSONDecodeError:
          print("Invalid JSON response:", response.text)
  else:
      print(f"Error: {response.status_code}, {response.text}")
      
  keywords = json.loads(response.json()['message']['content'])['keywords']
  
  return keywords


In [27]:
t = extract_keyword_by_llama32(query = 'How to become a software engineer')

{'model': 'llama3.2:latest', 'created_at': '2025-03-03T07:10:19.530484Z', 'message': {'role': 'assistant', 'content': '{ "keywords": ["software engineer", "become", "How to"] }'}, 'done_reason': 'stop', 'done': True, 'total_duration': 921803200, 'load_duration': 19863300, 'prompt_eval_count': 162, 'prompt_eval_duration': 43000000, 'eval_count': 18, 'eval_duration': 857000000}


In [29]:
t

['software engineer', 'become', 'How to']